## Estimate Trip arrival times

In [1]:
import pandas as pd

vehicle_positions = pd.read_csv("data/vehicle_tracking/processed/804_lametro-rail/2019-01-16.csv", index_col=0, parse_dates=["datetime"])
stations = pd.read_csv("data/line_info/804/804_directionA_stations.csv", index_col=0)

In [2]:
directionA = vehicle_positions[vehicle_positions['direction'] == 0]
trips = directionA[['datetime', 'trip_id', 'relative_position']].groupby(['trip_id'])
estimates = stations[['stop_id', 'relative_position']]
estimates['estimate'] = True

def create_estimated_arrival_times(trip_id, trip):
    trip['estimate'] = False # add field to indicate these are actual observations not estimates
    trip_est = estimates # copy values into loop variable
    trip_est['trip_id'] = trip_id
    combined = trip.append(trip_est)
    combined = combined.sort_values('relative_position')
    combined = combined.reset_index(drop=True)
    combined['previous_pos'] = combined.relative_position.shift() # shift vals to move adjacent position and date data into each row
    combined['next_pos'] = combined.relative_position.shift(-1)
    combined['previous_dt'] = combined.datetime.shift()
    combined['next_dt'] = combined.datetime.shift(-1)
    select = combined[combined['estimate']==True]
    select['weight'] = (select.relative_position - select.previous_pos) / (select.next_pos - select.previous_pos)
    select['time_interpolation'] = (select.next_dt - select.previous_dt) * select.weight
    select['datetime'] = select.previous_dt + select.time_interpolation
    select['datetime_round'] = pd.DatetimeIndex(select.datetime).round('S')
    select['stop_id'] = pd.to_numeric(select.stop_id, downcast='integer')
    return select

groups = [create_estimated_arrival_times(trip_id, trip) for trip_id, trip in trips]
"DONE"

/home/cts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/cts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/cts/venv/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the 

'DONE'

In [4]:
all_estimates = pd.concat(groups)
stop_estimates = all_estimates.groupby(['stop_id'])

## Assemble/parse raw predictions data

In [28]:
import json
import os
import pendulum
from helpers.datetimefs import DateTimeFS, construct_filename

agency = "lametro-rail"
now = pendulum.now("America/Los_Angeles")
today = "2019-01-16"
end_datetime = now

def process_frame(datetime, path_base):
    source_path = construct_filename(path_base, datetime, ".json")
    with open(source_path, "r") as infile:
        raw = json.load(infile)["predictions"]
    try:
        directionB = raw[0]["direction"]["prediction"]
    except:
        return None
    if type(directionB) is dict:
        df = pd.DataFrame(directionB, index=[0])
    elif type(directionB) is list:
        df = pd.DataFrame(directionB[0], index=[0]) # get first prediction only
    df["datetime_prediction_made"] = pd.to_datetime(datetime.to_iso8601_string())
    df["datetime_predicted"] = pd.DatetimeIndex(pd.to_datetime(pd.to_datetime(df["epochTime"], unit='ms'))).round('T')
    # predicted times are rounded to the minute, we are being generous/reasonable here
    return df
line_predictions = {}
for line in range(804, 805):
    schedule_path = f"data/schedule/{line}_{agency}/{today}.csv"
    schedule = pd.read_csv(schedule_path, index_col=0)
    start_datetime = pendulum.parse(
        schedule.datetime.min(), tz="America/Los_Angeles"
    )  
    stops = stations.stop_id
    stop_predictions = {}
    for stop_id in stops:
        path_base = f"data/predictions/raw/{line}_{agency}/{stop_id}"
        dtfs = DateTimeFS(path_base)
        datetimes = dtfs.get_datetimes_in_range(
            start_datetime.in_tz("UTC"), end_datetime.in_tz("UTC")
        )
        predictions = [process_frame(datetime, path_base) for datetime in datetimes]
        predictions = pd.concat(predictions).drop_duplicates(["dirTag", "datetime_predicted", "vehicle"])[["datetime_prediction_made", "datetime_predicted", "vehicle", "tripTag"]]
        stop_predictions[stop_id] = predictions
    line_predictions[line] = pd.concat(stop_predictions)
all_predictions = pd.concat(line_predictions)
"DONE"

/home/cts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



'DONE'

In [98]:
def match_predictions(key, group):
    est = stop_estimates.get_group(key[1]).dropna(subset=['datetime'])
    est = est.set_index(pd.DatetimeIndex(est['datetime'])).sort_index()
    if len(est):
        group.loc[:,"closest_actual"] = group.datetime_predicted.apply(lambda x: est.index[est.index.get_loc(x, method='nearest')])
        return group
    return None

comparable = pd.concat([match_predictions(key, group) for key, group in all_predictions.groupby(level=[0, 1])])

"DONE"

'DONE'

In [102]:
comparable.loc[(804)]

datetime_prediction_made  datetime_predicted vehicle   tripTag  \
80402 0      2019-01-16 12:04:10 2019-01-16 12:34:00     451  47674328   
      0      2019-01-16 12:31:22 2019-01-16 12:35:00     451  47674328   
      0      2019-01-16 12:35:09 2019-01-16 12:49:00     452  47674344   
      0      2019-01-16 12:49:07 2019-01-16 12:54:00     402  47674229   
      0      2019-01-16 12:53:09 2019-01-16 13:02:00     412  47674361   
      0      2019-01-16 13:00:09 2019-01-16 13:11:00     453  47674111   
      0      2019-01-16 13:09:09 2019-01-16 13:20:00     413  47674244   
      0      2019-01-16 13:20:12 2019-01-16 13:35:00     454  47674250   
      0      2019-01-16 13:35:10 2019-01-16 13:42:00     415  47674149   
      0      2019-01-16 13:42:13 2019-01-16 13:47:00     404  47674266   
      0      2019-01-16 13:47:14 2019-01-16 13:55:00     414  47674142   
      0      2019-01-16 13:52:14 2019-01-16 14:02:00     405  47674152   
      0      2019-01-16 13:59:07 2019-01-16 14:09:00     406  47674168   
      0      2019-01-16 14:08:07 2019-01-16 14:16:00     416  47674284   
      0      2019-01-16 14:13:07 2019-01-16 14:23:00     407  47674291   
      0      2019-01-16 14:21:08 2019-01-16 14:30:00     417  47674308   
      0      2019-01-16 14:29:08 2019-01-16 14:37:00     408  47674182   
      0      2019-01-16 14:35:07 2019-01-16 14:45:00     411  47674207   
      0      2019-01-16 14:44:08 2019-01-16 14:51:00     409  47674314   
      0      2019-01-16 14:48:08 2019-01-16 14:59:00     401  47674214   
      0      2019-01-16 14:59:07 2019-01-16 15:05:00     418  47674445   
      0      2019-01-16 15:05:08 2019-01-16 15:13:00     451  47674330   
      0      2019-01-16 15:13:08 2019-01-16 15:19:00     419  47674367   
      0      2019-01-16 15:19:08 2019-01-16 15:29:00     452  47674347   
      0      2019-01-16 15:25:07 2019-01-16 15:30:00     452  47674347   
      0      2019-01-16 15:29:16 2019-01-16 15:34:00     402  47674231   
      0      2019-01-16 15:30:08 2019-01-16 15:35:00     402  47674231   
      0      2019-01-16 15:35:08 2019-01-16 15:40:00     412  47674363   
      0      2019-01-16 15:40:08 2019-01-16 15:48:00     453  47674117   
      0      2019-01-16 15:48:08 2019-01-16 15:54:00     413  47674246   
...                          ...                 ...     ...       ...   
80427 0      2019-01-16 17:09:08 2019-01-16 17:10:00     453  47674123   
      0      2019-01-16 17:10:08 2019-01-16 17:15:00     413  47674247   
      0      2019-01-16 17:15:08 2019-01-16 17:37:00     415  47674140   
      0      2019-01-16 17:17:11 2019-01-16 17:36:00     415  47674140   
      0      2019-01-16 17:33:07 2019-01-16 17:34:00     415  47674140   
      0      2019-01-16 17:34:07 2019-01-16 17:43:00     404  47674269   
      0      2019-01-16 17:43:07 2019-01-16 17:50:00     414  47674144   
      0      2019-01-16 17:50:07 2019-01-16 17:51:00     414  47674144   
      0      2019-01-16 17:51:08 2019-01-16 17:57:00     405  47674155   
      0      2019-01-16 17:55:08 2019-01-16 17:56:00     405  47674155   
      0      2019-01-16 17:56:08 2019-01-16 18:08:00     406  47674171   
      0      2019-01-16 18:01:07 2019-01-16 18:07:00     406  47674171   
      0      2019-01-16 18:08:08 2019-01-16 18:11:00     416  47674287   
      0      2019-01-16 18:11:08 2019-01-16 18:19:00     407  47674293   
      0      2019-01-16 18:19:09 2019-01-16 18:28:00     417  47674311   
      0      2019-01-16 18:21:08 2019-01-16 18:31:00     417  47674311   
      0      2019-01-16 18:22:12 2019-01-16 18:33:00     417  47674311   
      0      2019-01-16 18:24:08 2019-01-16 18:34:00     417  47674311   
      0      2019-01-16 18:26:07 2019-01-16 18:35:00     408  47674185   
      0      2019-01-16 18:27:15 2019-01-16 18:39:00     408  47674185   
      0      2019-01-16 18:32:12 2019-01-16 18:38:00     408  47674185   
      0      2019-01-16 18:39:07 2019-01-16 18:42:00     411  47674209   
      0     